<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/time_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN(Recurrent Neural Network)


In [ ]:
class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        t = np.dot(h_prev, Wh) + np.dot(x, Wx) + b
        h_next = np.tanh(t)

        self.cache = (x, h_prev, h_next)
        return h_next

    def backward(self, dh_next): # dh_next는 흘러들어온 미분
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache

        dt = dh_next * (1 - h_next ** 2) # 활성화 함수 tanh에 대한 미분.(1-y^2)
        db = np.sum(dt, axis=0) # repeat node에 대한 역전파.
        dWh = np.dot(h_prev.T, dt) # dot 연산에 대한 역전파
        dh_prev = np.dot(dt, Wh.T) # dot 연산에 대한 역전파
        dWx = np.dot(x.T, dt) # dot 연산에 대한 역전파
        dx = np.dot(dt, Wx.T) # dot에 연산에 대한 역전파

        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

        return dx, dh_prev